In [1]:
import gensim.downloader as api
wv = api.load('word2vec-google-news-300')

In [2]:
wv.similarity(w1='great',w2='good')

0.729151

In [3]:
wv_great = wv['great']
wv_good = wv['good']

In [4]:
import pandas as pd

df = pd.read_csv('./Data/Fake_Real_Data.csv')

In [5]:
df.head()

,Text,label
0,Top Trump Surrogate BRUTALLY Stabs Him In The...,Fake
1,U.S. conservative leader optimistic of common ...,Real
2,"Trump proposes U.S. tax overhaul, stirs concer...",Real
3,Court Forces Ohio To Allow Millions Of Illega...,Fake
4,Democrats say Trump agrees to work on immigrat...,Real


In [6]:
df['label_num']=df['label'].map({'Fake':0,'Real':1})

In [7]:
import spacy
nlp = spacy.load('en_core_web_lg')
def preprocess_and_vectorize(text):
    doc = nlp(text)
    filtered_tokens=[]
    for token in doc:
        if token.is_punct or token.is_stop:
            continue
        filtered_tokens.append(token.lemma_)
    return wv.get_mean_vector(filtered_tokens)

In [8]:
preprocess_and_vectorize("don't worry if you don't have high score")

array([ 2.33738478e-02, -2.64660385e-03, -7.40598841e-03,  3.55755650e-02,
        4.87089157e-03, -1.50732296e-02,  3.90400365e-02, -4.58122790e-02,
        5.47190346e-02,  1.66951511e-02, -1.00811571e-02, -5.96696734e-02,
        3.46883051e-02, -7.52686709e-03, -2.01580557e-03,  1.47714810e-02,
        2.11633071e-02,  6.96659163e-02, -2.35075261e-02, -6.63120672e-02,
       -2.11626515e-02,  3.19983736e-02,  3.30468193e-02,  3.13876010e-02,
        4.91791479e-02, -6.96077058e-03, -2.07300950e-02, -5.25327884e-02,
       -5.25226444e-03, -6.82936907e-02, -7.13493908e-03, -3.00915837e-02,
        7.62140676e-02, -5.23094051e-02, -2.84775738e-02, -1.11878766e-02,
       -1.99600626e-02,  2.46490538e-02,  2.67670695e-02,  5.15096150e-02,
        5.64508475e-02, -1.92053057e-02,  5.05131893e-02, -1.73199195e-02,
       -4.66615856e-02,  9.27811954e-03,  7.45154591e-03,  3.26672085e-02,
        2.65876353e-02,  4.23127003e-02, -7.28106825e-03,  4.77878638e-02,
       -1.51969157e-02, -

In [9]:
wv.get_mean_vector(['worry','understand']).shape

(300,)

In [10]:
df['vector'] = df["Text"].apply(lambda text:preprocess_and_vectorize(text))

In [12]:
from sklearn.model_selection import train_test_split
import numpy as np

#Do the 'train-test' splitting with test size of 20% with random state of 2022 and stratify sampling too
X_train, X_test, y_train, y_test = train_test_split(
    df.vector.values, 
    df.label_num, 
    test_size=0.2, # 20% samples will go to test dataset
    random_state=2022,
    stratify=df.label_num
)
print("Shape of X_train before reshaping: ", X_train.shape)
print("Shape of X_test before reshaping: ", X_test.shape)


X_train_2d = np.stack(X_train)
X_test_2d =  np.stack(X_test)

print("Shape of X_train after reshaping: ", X_train_2d.shape)
print("Shape of X_test after reshaping: ", X_test_2d.shape)

Shape of X_train before reshaping:  (7920,)
Shape of X_test before reshaping:  (1980,)
Shape of X_train after reshaping:  (7920, 300)
Shape of X_test after reshaping:  (1980, 300)


In [13]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import classification_report

#1. creating a GradientBoosting model object
clf = GradientBoostingClassifier()

#2. fit with all_train_embeddings and y_train
clf.fit(X_train_2d, y_train)


#3. get the predictions for all_test_embeddings and store it in y_pred
y_pred = clf.predict(X_test_2d)


#4. print the classfication report
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.98      0.97      0.98      1000
           1       0.97      0.98      0.98       980

    accuracy                           0.98      1980
   macro avg       0.98      0.98      0.98      1980
weighted avg       0.98      0.98      0.98      1980

